In [41]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

In [42]:
class WordList(BaseModel):
    """A list of random words.
    """
    words: List[str] = Field(description="A list of words")
    tag: Optional[str] = Field(description="A tag for the list of words")

In [71]:
def get_words_prompt(tag: Optional[str]):
    prompt = "Generate {num_words} random words that are about a {vocab_level} reading level."
    if tag:
        prompt += "\n The words should be related to {tag}. Include the tag in the output."

    return prompt


In [88]:
import json
import dotenv
from langchain_core.output_parsers import StrOutputParser

#from langchain_core.callbacks import CallbackHandler


dotenv.load_dotenv()
tag = "how it feels to be in a place"
prompt_str = get_words_prompt(tag)
print(prompt_str)
word_gen_prompt = PromptTemplate.from_template(
    template=prompt_str,
)

word_gen_model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1, max_tokens=1000)

word_gen_model_with_tools = word_gen_model.with_structured_output(WordList)

word_gen_chain = word_gen_prompt | word_gen_model_with_tools
word_list = word_gen_chain.invoke({"num_words": 100, "vocab_level": "third grade", "tag": tag})

print(word_list)
# Specify the file path where you want to save the JSON file
file_path = 'word_list.json'

# Load existing data
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Your new data
new_data = word_list.dict()

# Append new data
data.append(new_data)

# Write back to file
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)


Generate {num_words} random words that are about a {vocab_level} reading level.
 The words should be related to {tag}. Include the tag in the output.
words=['happy', 'uncomfortable', 'relaxed', 'excited', 'peaceful', 'nervous', 'curious', 'scared', 'content', 'bored'] tag='Feelings in a Place'


In [ ]:
import json

# Specify the file path of the JSON file
file_path = 'word_list.json'

# Read the JSON file
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

new_file_path = "aggregate_words.json"
new_file_data = {}

# Iterate through the items
for item in data:
    # Placeholder for business logic handling each item
    # Add your code here to handle each item in the JSON file
    tag = item['tag']
    for word in item['words']:
        if new_file_data.get(word):
            new_tags = new_file_data[word]['tags'] + [tag] if tag else new_file_data[word]['tags']
            new_tags = list(set(new_tags))
            new_file_data[word].update({
                "count": new_file_data[word]['count'] + 1,
                "tags": new_tags
            })
            # print("updated a word..", new_file_data[word])
        else:
            tags = [tag] if tag else []
            new_file_data[word] = {
                "count": 1,
                "tags": tags
            }
            # print("added a new word.", new_file_data[word])

# Write the new file data to the JSON file
with open(new_file_path, 'w') as json_file:
    json.dump(new_file_data, json_file, indent=4)



In [101]:
print(len(new_file_data.keys()))

724
